# Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews <br>

EDA: https://nycdatascience.com/blog/student-works/amazon-fine-foods-visualization/


The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.




In [1]:
from scipy.sparse import csr_matrix

lst = [1, 2, 3, 4, 5, 6, 7]

ans = sparse.csr_matrix(lst)

NameError: name 'sparse' is not defined

## Loading the data

The dataset is available in two forms
1. .csv file
2. SQLite Database

In order to load the data, We have used the SQLITE dataset as it easier to query the data and visualise the data efficiently.
<br> 

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score id above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

unable to import 'smart_open.gcs', disabling that module


# [1]. Reading Data

In [5]:
#establishing the connection
conn = sqlite3.connect('database.sqlite')

In [6]:
filtered_data=pd.read_sql_query("select * from Reviews WHERE Score!=3 limit 5000",conn)
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [9]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", conn)

In [10]:
print(display.shape)
display.head()

(80668, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [11]:
display[display['UserId']=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
80638,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",1296691200,5,I bought this 6 pack because for the price tha...,5


In [12]:
display['COUNT(*)'].sum()

393063

#  Exploratory Data Analysis

## [2] Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.  Following is an example:

In [13]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", conn)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text  and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [14]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [15]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(4986, 10)

In [16]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

99.72

<b>Observation:-</b> It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [17]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", conn)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [18]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [19]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(4986, 10)


5    3412
4     766
1     502
2     306
Name: Score, dtype: int64

# [3].  Text Preprocessing.

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [20]:
# printing some random reviews
sent_0 = final['Summary'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Summary'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Summary'].values[1500]
print(sent_1500)
print("="*50)

sent_4900 = final['Summary'].values[4900]
print(sent_4900)
print("="*50)

thirty bucks?
Best sour cream & onion chip I've had
Are We Reviewing Our Mistakes Or These Cookies?
caribou


In [21]:
# remove urls from text python: https://stackoverflow.com/a/40823105/4084039
sent_0 = re.sub(r"http\S+", "", sent_0)
sent_1000 = re.sub(r"http\S+", "", sent_1000)
sent_150 = re.sub(r"http\S+", "", sent_1500)
sent_4900 = re.sub(r"http\S+", "", sent_4900)

print(sent_0)

thirty bucks?


In [22]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup


soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)


soup = BeautifulSoup(sent_1000, 'lxml')
text = soup.get_text()
print(text)
print("="*50)


soup = BeautifulSoup(sent_1500, 'lxml')
text = soup.get_text()
print(text)
print("="*50)


soup = BeautifulSoup(sent_4900, 'lxml')
text = soup.get_text()
print(text)

thirty bucks?
Best sour cream & onion chip I've had
Are We Reviewing Our Mistakes Or These Cookies?
caribou


In [23]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [24]:
sent_1500 = decontracted(sent_1500)
print(sent_1500)
print("="*50)

Are We Reviewing Our Mistakes Or These Cookies?


In [25]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

thirty bucks?


In [26]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
print(sent_1500)

Are We Reviewing Our Mistakes Or These Cookies 


In [27]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [28]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:04<00:00, 1055.42it/s]


In [29]:
preprocessed_reviews[1500]

'wow far two two star reviews one obviously no idea ordering wants crispy cookies hey sorry reviews nobody good beyond reminding us look ordering chocolate oatmeal cookies not like combination not order type cookie find combo quite nice really oatmeal sort calms rich chocolate flavor gives cookie sort coconut type consistency let also remember tastes differ given opinion soft chewy cookies advertised not crispy cookies blurb would say crispy rather chewy happen like raw cookie dough however not see taste like raw cookie dough soft however confusion yes stick together soft cookies tend not individually wrapped would add cost oh yeah chocolate chip cookies tend somewhat sweet want something hard crisp suggest nabiso ginger snaps want cookie soft chewy tastes like combination chocolate oatmeal give try place second order'

<h2><font color='red'>[3.2] Preprocess Summary</font></h2>

In [30]:
## Similartly you can do preprocessing for review summary also.


#checking for any 
for count,i in enumerate(final['Summary']):
    a=re.search(r"http\S+",i)
    if a!=None:
        print(count, a)

# printing some random Summary

#for count,i in enumerate(final['Summary']):
#    a=re.search(r'[^A-Za-z0-9]+',i)
#    if a!=None:
#        print(count, a)

print("\n******* Summaries with numbers***********\n")
    
summary_292 = final['Summary'].values[292]
print(summary_292)
print("="*50)

summary_345  = final['Summary'].values[345]
print(summary_345)
print("="*50)

summary_3073= final['Summary'].values[3073]
print(summary_3073)
print("="*50)

print("\n******* Summaries with numbers***********\n")

#checking for numbers

#for count,i in enumerate(final['Summary']):
#    a=re.search(r'\S*\d\S*',i)
#    if a!=None:
#        print(count, a)


summary_530 =final['Summary'].values[530]
print(summary_530)
print("="*50)

summary_174 = final['Summary'].values[174]
print(summary_174)
print("="*50)

summary_4222= final['Summary'].values[4222]
print(summary_4222)
print("="*50)




******* Summaries with numbers***********

Yummy!
cook & serve choc. fudge pudding
Acceptable & inexpensive, but GREAT when cold brewed!

******* Summaries with numbers***********

50 calories of yumminess
White Stevia 12 Oz Pwdr
KEEBLER SOFT BATCH CHOCOLATE CHIP COOKIES 15-OUNCE(PACK OF 6)


## Summary: 

1.No Hyperlinks are present

2.found Special charecters and Numbers(digits)

In [31]:
Preprocessed_Summary=[]
for summary in tqdm(final['Summary'].values): 
    summary = decontracted(summary)
    summary = re.sub("\S*\d\S*", " ", summary).strip()
    summary=re.sub(r'[^A-Za-z0-9]+',' ',summary)
    summ=' '.join([i.lower() for i in summary.split() if i.lower() not in stopwords])
    Preprocessed_Summary.append(summ.strip())
print(Preprocessed_Summary)

100%|███████████████████████████████████████████████████████████████████████████| 4986/4986 [00:00<00:00, 24946.71it/s]


['thirty bucks', 'flies begone', 'wow make islickers', 'great product', 'good stuff', 'premium quality dog food', 'cats love', 'nice big pieces big almond flavor', 'summer treat fat free guilt free', 'not buy product unless looking shredded coconut', 'little flavor', 'staple house', 'favorite quick meal solution', 'best hot sauce taco sauce available america', 'pico pica best', 'stuff', 'everyone saying pico pica true', 'not edible', 'shining star', 'inexpensive alternative gold leaf', 'create exquisite cake decorations', 'gold dust awesome', 'perfect sons cake', 'really cute made great golf cake', 'golf set', 'cake topper', 'fantastic', 'adzuki beans', 'yum', 'yummy', 'beans', 'fabulous', 'awesome', 'mae ploy chili sauce', 'move outta way ketchup barbecue sauce', 'mae ploy sauce great stuff', 'oh', 'pei wei pf changs house', 'great sauce', 'guess minority', 'classic condiment', 'highly addicting', 'put meats', 'love', 'yum', 'yummy sauce', 'great sauce', 'restaurants serve', 'favorite

# [4] Featurization

## [4.1] BAG OF WORDS

In [32]:
#BoW
count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(preprocessed_reviews)
print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

final_counts = count_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

some feature names  ['aa', 'aahhhs', 'aback', 'abandon', 'abates', 'abbott', 'abby', 'abdominal', 'abiding', 'ability']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (4986, 12997)
the number of unique words  12997


## [4.2] Bi-Grams and n-Grams.

In [33]:
#bi-gram, tri-gram and n-gram

#removing stop words like "not" should be avoided before building n-grams
# count_vect = CountVectorizer(ngram_range=(1,2))
# please do read the CountVectorizer documentation http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# you can choose these numebrs min_df=10, max_features=5000, of your choice
count_vect = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
final_bigram_counts = count_vect.fit_transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (4986, 3144)
the number of unique words including both unigrams and bigrams  3144


## [4.3] TF-IDF

In [34]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vect.fit(preprocessed_reviews)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

final_tf_idf = tf_idf_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

some sample features(unique words in the corpus) ['ability', 'able', 'able find', 'able get', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (4986, 3144)
the number of unique words including both unigrams and bigrams  3144


## [4.4] Word2Vec

In [35]:
# Train your own Word2Vec model using your own text corpus
i=0
list_of_sentance=[]
for sentance in preprocessed_reviews:
    list_of_sentance.append(sentance.split())

In [36]:
# Using Google News Word2Vectors

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.


# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W17SRFAzZPY
# you can comment this whole cell
# or change these varible according to your need

is_your_ram_gt_16g=False
want_to_use_google_w2v = False
want_to_train_w2v = True

if want_to_train_w2v:
    # min_count = 5 considers only words that occured atleast 5 times
    w2v_model=Word2Vec(list_of_sentance,min_count=5,size=50, workers=4)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))
    
elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('GoogleNews-vectors-negative300.bin'):
        w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print(w2v_model.wv.most_similar('great'))
        print(w2v_model.wv.most_similar('worst'))
    else:
        print("you don't have gogole's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")

[('want', 0.9948499202728271), ('enjoy', 0.993760347366333), ('think', 0.9931143522262573), ('snack', 0.9930903315544128), ('flavorful', 0.9925112128257751), ('tasty', 0.992412805557251), ('heaven', 0.9923456907272339), ('overpowering', 0.9921022653579712), ('salty', 0.9920707941055298), ('tasting', 0.9920499324798584)]
[('varieties', 0.9994068145751953), ('berry', 0.9993639588356018), ('popcorn', 0.9992774724960327), ('type', 0.9992654323577881), ('hands', 0.9992233514785767), ('wow', 0.9992210268974304), ('somewhat', 0.9992208480834961), ('come', 0.9992064833641052), ('tassimo', 0.9991840720176697), ('mccann', 0.999176025390625)]


In [37]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  3817
sample words  ['product', 'available', 'course', 'total', 'pretty', 'stinky', 'right', 'nearby', 'used', 'ca', 'not', 'beat', 'great', 'received', 'shipment', 'could', 'hardly', 'wait', 'try', 'love', 'call', 'instead', 'removed', 'easily', 'daughter', 'designed', 'printed', 'use', 'car', 'windows', 'beautifully', 'shop', 'program', 'going', 'lot', 'fun', 'everywhere', 'like', 'tv', 'computer', 'really', 'good', 'idea', 'final', 'outstanding', 'window', 'everybody', 'asks', 'bought', 'made']


## [4.4.1] Converting text into vectors using wAvg W2V, TFIDF-W2V

#### [4.4.1.1] Avg W2v

In [76]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

100%|████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:04<00:00, 1024.03it/s]


4986
50


#### [4.4.1.2] TFIDF weighted W2v

In [39]:
# S = ["abc def pqr", "def def def abc", "pqr pqr def"]
model = TfidfVectorizer()
model.fit(preprocessed_reviews)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [40]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|█████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:31<00:00, 156.58it/s]


# [5] Applying TSNE

ol>
    <li><strong>Apply Knn(brute force version) on these feature sets</strong>
        <ul>
            <li><font color='red'>SET 1:</font>Review text, preprocessed one converted into vectors using (BOW)</li>
            <li><font color='red'>SET 2:</font>Review text, preprocessed one converted into vectors using (TFIDF)</li>
            <li><font color='red'>SET 3:</font>Review text, preprocessed one converted into vectors using (AVG W2v)</li>
            <li><font color='red'>SET 4:</font>Review text, preprocessed one converted into vectors using (TFIDF W2v)</li>
        </ul>
    </li>
    <br>
    <li><strong>Apply Knn(kd tree version) on these feature sets</strong>
        <br><font color='red'>NOTE: </font>sklearn implementation of kd-tree accepts only dense matrices, you need to convert the sparse matrices of CountVectorizer/TfidfVectorizer into dense matices. You can convert sparse matrices to dense using .toarray() attribute. For more information please visit this <a href='https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.toarray.html'>link</a>
        <ul>
            <li><font color='red'>SET 5:</font>Review text, preprocessed one converted into vectors using (BOW) but with restriction on maximum features generated.
            <pre>
            count_vect = CountVectorizer(min_df=10, max_features=500) 
            count_vect.fit(preprocessed_reviews)
            </pre>
            </li>
            <li><font color='red'>SET 6:</font>Review text, preprocessed one converted into vectors using (TFIDF) but with restriction on maximum features generated.
            <pre>
                tf_idf_vect = TfidfVectorizer(min_df=10, max_features=500)
                tf_idf_vect.fit(preprocessed_reviews)
            </pre>
            </li>
            <li><font color='red'>SET 3:</font>Review text, preprocessed one converted into vectors using (AVG W2v)</li>
            <li><font color='red'>SET 4:</font>Review text, preprocessed one converted into vectors using (TFIDF W2v)</li>
        </ul>
    </li>
    <br>
    <li><strong>The hyper paramter tuning(find best K)</strong>
        <ul>
    <li>Find the best hyper parameter which will give the maximum <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/receiver-operating-characteristic-curve-roc-curve-and-auc-1/'>AUC</a> value</li>
    <li>Find the best hyper paramter using k-fold cross validation or simple cross validation data</li>
    <li>Use gridsearch cv or randomsearch cv or you can also write your own for loops to do this task of hyperparameter tuning</li>
        </ul>
    </li>
    <br>
    <li>
    <strong>Representation of results</strong>
        <ul>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='train_cv_auc.JPG' width=300px></li>
    <li>Once after you found the best hyper parameter, you need to train your model with it, and find the AUC on test data and plot the ROC curve on both train and test.
    <img src='train_test_auc.JPG' width=300px></li>
    <li>Along with plotting ROC curve, you need to print the <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/confusion-matrix-tpr-fpr-fnr-tnr-1/'>confusion matrix</a> with predicted and original labels of test data points
    <img src='confusion_matrix.png' width=300px></li>
        </ul>
    </li>
    <br>
    <li><strong>Conclusion</strong>
        <ul>
    <li>You need to summarize the results at the end of the notebook, summarize it in the table format. To print out a table please refer to this prettytable library<a href='http://zetcode.com/python/prettytable/'> link</a> 
        <img src='summary.JPG' width=400px>
    </li>
        </ul>
</ol>

<h4><font color='red'>Note: Data Leakage</font></h4>

1. There will be an issue of data-leakage if you vectorize the entire data and then split it into train/cv/test.
2. To avoid the issue of data-leakag, make sure to split your data first and then vectorize it. 
3. While vectorizing your data, apply the method fit_transform() on you train data, and apply the method transform() on cv/test data.
4. For more details please go through this <a href='https://soundcloud.com/applied-ai-course/leakage-bow-and-tfidf'>link.</a>

# [5.1] Applying KNN brute force

### [5.1.1] Applying KNN brute force on BOW,<font color='red'> SET 1</font>

In [97]:
import scipy
#converting the matrix to densematrix
final_dense=final_counts.todense()#(4986, 12997)
print(final_dense.shape)
x=final_dense


#converting score into Postive and negative review
def df_score(x):
    if x<3:
        return 0
    else:
        return 1
    
y = final['Score'].apply(df_score)

y=y.values

(4986, 12997)


In [98]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


#spliting the data into train,cv and test. 
x_train,x_1,y_train,y_1=train_test_split(x,y,train_size =0.6,random_state=0)
x_cv,x_test,y_cv,y_test=train_test_split(x_1,y_1,train_size =0.5,random_state=0)


#priting the rows against train,cv and test
print("x_train_shape : ", x_train.shape[0])
print("y_train_shape : ", y_train.shape[0])
print("x_cv_shape : ", x_cv.shape[0])
print("y_cv_shape : ", y_cv.shape[0])
print("x_test_shape : ", x_test.shape[0])
print("y_test_shape : ", y_test.shape[0])

x_train_shape :  2991
y_train_shape :  2991
x_cv_shape :  997
y_cv_shape :  997
x_test_shape :  998
y_test_shape :  998


In [99]:
CV_Accuracy=dict()#making the dict. 
for k in range(1,30):
    knn=KNeighborsClassifier(n_neighbors=k,algorithm='brute')#calling the Knn classifiers. 
    knn.fit(x_train,y_train)#fiting with Dtrain data 
    pred=knn.predict(x_cv)#predicting the label values with CV data
    acc=accuracy_score(y_cv,pred)*float(100) #finding the accurarcy.
    CV_Accuracy[k]=acc#storing the key valye pairs of K and accuracy. 
    
#getting the optimal Value    
optimal_k=max(CV_Accuracy,key=CV_Accuracy.get)

#calling the Knn classifier with optimal value.
knn=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='brute')
knn.fit(x_train,y_train)#fiting with Dtest data
pred=knn.predict(x_test)#predicting the label values with Dtest
acc=accuracy_score(y_test,pred)*float(100)#finding the accuracy. 
print("The accuracy of the model for Cross validation optimal-K: {0} is {1} ".format(optimal_k,acc))

The accuracy of the model for Cross validation optimal-K: 9 is 85.37074148296593 


### [5.1.2] Applying KNN brute force on TFIDF,<font color='red'> SET 2</font>

In [92]:
import scipy
#converting the matrix to densematrix
final_dense=final_tf_idf.todense()#(4986, 12997)
print(final_dense.shape)
x=final_dense


#converting score into Postive and negative review
def df_score(x):
    if x<3:
        return 0
    else:
        return 1
    
y = final['Score'].apply(df_score)

y=y.values

(4986, 3144)


In [93]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


#spliting the data into train,cv and test. 
x_train,x_1,y_train,y_1=train_test_split(x,y,train_size =0.6,random_state=0)
x_cv,x_test,y_cv,y_test=train_test_split(x_1,y_1,train_size =0.5,random_state=0)


#priting the rows against train,cv and test
print("x_train_shape : ", x_train.shape[0])
print("y_train_shape : ", y_train.shape[0])
print("x_cv_shape : ", x_cv.shape[0])
print("y_cv_shape : ", y_cv.shape[0])
print("x_test_shape : ", x_test.shape[0])
print("y_test_shape : ", y_test.shape[0])

x_train_shape :  2991
y_train_shape :  2991
x_cv_shape :  997
y_cv_shape :  997
x_test_shape :  998
y_test_shape :  998


In [95]:
CV_Accuracy=dict()#making the dict. 
for k in range(1,30):
    knn=KNeighborsClassifier(n_neighbors=k,algorithm='brute')#calling the Knn classifiers. 
    knn.fit(x_train,y_train)#fiting with Dtrain data 
    pred=knn.predict(x_cv)#predicting the label values with CV data
    acc=accuracy_score(y_cv,pred)*float(100) #finding the accurarcy.
    CV_Accuracy[k]=acc#storing the key valye pairs of K and accuracy. 
    
#getting the optimal Value    
optimal_k=max(CV_Accuracy,key=CV_Accuracy.get)

#calling the Knn classifier with optimal value.
knn=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='brute')
knn.fit(x_train,y_train)#fiting with Dtest data
pred=knn.predict(x_test)#predicting the label values with Dtest
acc=accuracy_score(y_test,pred)*float(100)#finding the accuracy. 
print("The accuracy of the model for Cross validation optimal-K: {0} is {1} ".format(optimal_k,acc))

The accuracy of the model for Cross validation optimal-K: 5 is 85.07014028056112 


### [5.1.3] Applying KNN brute force on AVG W2V,<font color='red'> SET 3</font>

In [84]:
import numpy as np
#converting the matrix to densematrix
x = sent_vectors

#converting score into Postive and negative review
def df_score(x):
    if x<3:
        return 0
    else:
        return 1
    
y = final['Score'].apply(df_score)

y=y.values

In [101]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


#spliting the data into train,cv and test. 
x_train,x_1,y_train,y_1=train_test_split(x,y,train_size =0.6,random_state=0)
x_cv,x_test,y_cv,y_test=train_test_split(x_1,y_1,train_size =0.5,random_state=0)


#priting the rows against train,cv and test
print("x_train_total rows : ", len(x_train))
print("y_train_total rows : ", len(y_train))
print("x_cv_total rows : ", len(x_cv))
print("y_cv_total rows : ", len(y_cv))
print("x_test_total rows : ", len(x_test))
print("y_test_total rows : ", len(y_test))

x_train_total rows :  2991
y_train_total rows :  2991
x_cv_total rows :  997
y_cv_total rows :  997
x_test_total rows :  998
y_test_total rows :  998


In [90]:
CV_Accuracy=dict()#making the dict. 
for k in range(1,30):
    knn=KNeighborsClassifier(n_neighbors=k,algorithm='brute')#calling the Knn classifiers. 
    knn.fit(x_train,y_train)#fiting with Dtrain data 
    pred=knn.predict(x_cv)#predicting the label values with CV data
    acc=accuracy_score(y_cv,pred)*float(100) #finding the accurarcy.
    CV_Accuracy[k]=acc#storing the key valye pairs of K and accuracy. 
    
#getting the optimal Value    
optimal_k=max(CV_Accuracy,key=CV_Accuracy.get)

#calling the Knn classifier with optimal value.
knn=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='brute')
knn.fit(x_train,y_train)#fiting with Dtest data
pred=knn.predict(x_test)#predicting the label values with Dtest
acc=accuracy_score(y_test,pred)*float(100)#finding the accuracy. 
print("The accuracy of the model for Cross validation optimal-K {0} is {1} ".format(optimal_k,acc))

The accuracy of the model for Cross validation optimal-K 19 is 84.66933867735472 


### [5.1.4] Applying KNN brute force on TFIDF W2V,<font color='red'> SET 4</font>

In [102]:
import numpy as np
#converting the matrix to densematrix
x = tfidf_sent_vectors

#converting score into Postive and negative review
def df_score(x):
    if x<3:
        return 0
    else:
        return 1
    
y = final['Score'].apply(df_score)

y=y.values

In [104]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


#spliting the data into train,cv and test. 
x_train,x_1,y_train,y_1=train_test_split(x,y,train_size =0.6,random_state=0)
x_cv,x_test,y_cv,y_test=train_test_split(x_1,y_1,train_size =0.5,random_state=0)


#priting the rows against train,cv and test
print("x_train_total rows : ", len(x_train))
print("y_train_total rows : ", len(y_train))
print("x_cv_total rows : ", len(x_cv))
print("y_cv_total rows : ", len(y_cv))
print("x_test_total rows : ", len(x_test))
print("y_test_total rows : ", len(y_test))

x_train_total rows :  2991
y_train_total rows :  2991
x_cv_total rows :  997
y_cv_total rows :  997
x_test_total rows :  998
y_test_total rows :  998


In [105]:
CV_Accuracy=dict()#making the dict. 
for k in range(1,30):
    knn=KNeighborsClassifier(n_neighbors=k,algorithm='brute')#calling the Knn classifiers. 
    knn.fit(x_train,y_train)#fiting with Dtrain data 
    pred=knn.predict(x_cv)#predicting the label values with CV data
    acc=accuracy_score(y_cv,pred)*float(100) #finding the accurarcy.
    CV_Accuracy[k]=acc#storing the key valye pairs of K and accuracy. 
    
#getting the optimal Value    
optimal_k=max(CV_Accuracy,key=CV_Accuracy.get)

#calling the Knn classifier with optimal value.
knn=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='brute')
knn.fit(x_train,y_train)#fiting with Dtest data
pred=knn.predict(x_test)#predicting the label values with Dtest
acc=accuracy_score(y_test,pred)*float(100)#finding the accuracy. 
print("The accuracy of the model for Cross validation optimal-K {0} is {1} ".format(optimal_k,acc))

The accuracy of the model for Cross validation optimal-K 13 is 84.96993987975952 


## [5.2] Applying KNN kd-tree

### [5.2.1] Applying KNN kd-tree on BOW,<font color='red'> SET 5</font>

In [112]:
import scipy
#converting the matrix to densematrix
final_dense=final_counts.todense()#(4986, 12997)
print(final_dense.shape)
x=final_dense


#converting score into Postive and negative review
def df_score(x):
    if x<3:
        return 0
    else:
        return 1
    
y = final['Score'].apply(df_score)

y=y.values

(4986, 12997)


In [113]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


#spliting the data into train,cv and test. 
x_train,x_1,y_train,y_1=train_test_split(x,y,train_size =0.6,random_state=0)
x_cv,x_test,y_cv,y_test=train_test_split(x_1,y_1,train_size =0.5,random_state=0)


#priting the rows against train,cv and test
print("x_train_shape : ", x_train.shape[0])
print("y_train_shape : ", y_train.shape[0])
print("x_cv_shape : ", x_cv.shape[0])
print("y_cv_shape : ", y_cv.shape[0])
print("x_test_shape : ", x_test.shape[0])
print("y_test_shape : ", y_test.shape[0])

x_train_shape :  2991
y_train_shape :  2991
x_cv_shape :  997
y_cv_shape :  997
x_test_shape :  998
y_test_shape :  998


In [114]:
CV_Accuracy=dict()#making the dict. 
for k in range(1,20):
    knn=KNeighborsClassifier(n_neighbors=k,algorithm='kd_tree')#calling the Knn classifiers. 
    knn.fit(x_train,y_train)#fiting with Dtrain data 
    pred=knn.predict(x_cv)#predicting the label values with CV data
    acc=accuracy_score(y_cv,pred)*float(100) #finding the accurarcy.
    CV_Accuracy[k]=acc#storing the key valye pairs of K and accuracy. 
    
#getting the optimal Value    
optimal_k=max(CV_Accuracy,key=CV_Accuracy.get)

#calling the Knn classifier with optimal value.
knn=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='kd_tree')
knn.fit(x_train,y_train)#fiting with Dtest data
pred=knn.predict(x_test)#predicting the label values with Dtest
acc=accuracy_score(y_test,pred)*float(100)#finding the accuracy. 
print("The accuracy of the model for Cross validation optimal-K: {0} is {1} ".format(optimal_k,acc))

The accuracy of the model for Cross validation optimal-K: 7 is 84.5691382765531 


### [5.2.2] Applying KNN kd-tree on TFIDF,<font color='red'> SET 6</font>

In [115]:
import scipy
#converting the matrix to densematrix
final_dense=final_tf_idf.todense()#(4986, 12997)
print(final_dense.shape)
x=final_dense


#converting score into Postive and negative review
def df_score(x):
    if x<3:
        return 0
    else:
        return 1
    
y = final['Score'].apply(df_score)

y=y.values

(4986, 3144)


In [116]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


#spliting the data into train,cv and test. 
x_train,x_1,y_train,y_1=train_test_split(x,y,train_size =0.6,random_state=0)
x_cv,x_test,y_cv,y_test=train_test_split(x_1,y_1,train_size =0.5,random_state=0)


#priting the rows against train,cv and test
print("x_train_shape : ", x_train.shape[0])
print("y_train_shape : ", y_train.shape[0])
print("x_cv_shape : ", x_cv.shape[0])
print("y_cv_shape : ", y_cv.shape[0])
print("x_test_shape : ", x_test.shape[0])
print("y_test_shape : ", y_test.shape[0])

x_train_shape :  2991
y_train_shape :  2991
x_cv_shape :  997
y_cv_shape :  997
x_test_shape :  998
y_test_shape :  998


In [117]:
CV_Accuracy=dict()#making the dict. 
for k in range(1,20):
    knn=KNeighborsClassifier(n_neighbors=k,algorithm='kd_tree')#calling the Knn classifiers. 
    knn.fit(x_train,y_train)#fiting with Dtrain data 
    pred=knn.predict(x_cv)#predicting the label values with CV data
    acc=accuracy_score(y_cv,pred)*float(100) #finding the accurarcy.
    CV_Accuracy[k]=acc#storing the key valye pairs of K and accuracy. 
    
#getting the optimal Value    
optimal_k=max(CV_Accuracy,key=CV_Accuracy.get)

#calling the Knn classifier with optimal value.
knn=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='kd_tree')
knn.fit(x_train,y_train)#fiting with Dtest data
pred=knn.predict(x_test)#predicting the label values with Dtest
acc=accuracy_score(y_test,pred)*float(100)#finding the accuracy. 
print("The accuracy of the model for Cross validation optimal-K: {0} is {1} ".format(optimal_k,acc))

The accuracy of the model for Cross validation optimal-K: 5 is 85.07014028056112 


### [5.2.3] Applying KNN kd-tree on AVG W2V,<font color='red'> SET 7</font>

In [118]:
import numpy as np
#converting the matrix to densematrix
x = sent_vectors

#converting score into Postive and negative review
def df_score(x):
    if x<3:
        return 0
    else:
        return 1
    
y = final['Score'].apply(df_score)

y=y.values

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


#spliting the data into train,cv and test. 
x_train,x_1,y_train,y_1=train_test_split(x,y,train_size =0.6,random_state=0)
x_cv,x_test,y_cv,y_test=train_test_split(x_1,y_1,train_size =0.5,random_state=0)


#priting the rows against train,cv and test
print("x_train_total rows : ", len(x_train))
print("y_train_total rows : ", len(y_train))
print("x_cv_total rows : ", len(x_cv))
print("y_cv_total rows : ", len(y_cv))
print("x_test_total rows : ", len(x_test))
print("y_test_total rows : ", len(y_test))

x_train_total rows :  2991
y_train_total rows :  2991
x_cv_total rows :  997
y_cv_total rows :  997
x_test_total rows :  998
y_test_total rows :  998


In [120]:
CV_Accuracy=dict()#making the dict. 
for k in range(1,30):
    knn=KNeighborsClassifier(n_neighbors=k,algorithm='kd_tree')#calling the Knn classifiers. 
    knn.fit(x_train,y_train)#fiting with Dtrain data 
    pred=knn.predict(x_cv)#predicting the label values with CV data
    acc=accuracy_score(y_cv,pred)*float(100) #finding the accurarcy.
    CV_Accuracy[k]=acc#storing the key valye pairs of K and accuracy. 
    
#getting the optimal Value    
optimal_k=max(CV_Accuracy,key=CV_Accuracy.get)

#calling the Knn classifier with optimal value.
knn=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='kd_tree')
knn.fit(x_train,y_train)#fiting with Dtest data
pred=knn.predict(x_test)#predicting the label values with Dtest
acc=accuracy_score(y_test,pred)*float(100)#finding the accuracy. 
print("The accuracy of the model for Cross validation optimal-K {0} is {1} ".format(optimal_k,acc))

The accuracy of the model for Cross validation optimal-K 19 is 84.66933867735472 


### [5.2.4] Applying KNN kd-tree on TFIDF W2V,<font color='red'> SET 4</font>

In [122]:
import numpy as np
#converting the matrix to densematrix
x = tfidf_sent_vectors

#converting score into Postive and negative review
def df_score(x):
    if x<3:
        return 0
    else:
        return 1
    
y = final['Score'].apply(df_score)

y=y.values

In [123]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


#spliting the data into train,cv and test. 
x_train,x_1,y_train,y_1=train_test_split(x,y,train_size =0.6,random_state=0)
x_cv,x_test,y_cv,y_test=train_test_split(x_1,y_1,train_size =0.5,random_state=0)


#priting the rows against train,cv and test
print("x_train_total rows : ", len(x_train))
print("y_train_total rows : ", len(y_train))
print("x_cv_total rows : ", len(x_cv))
print("y_cv_total rows : ", len(y_cv))
print("x_test_total rows : ", len(x_test))
print("y_test_total rows : ", len(y_test))

x_train_total rows :  2991
y_train_total rows :  2991
x_cv_total rows :  997
y_cv_total rows :  997
x_test_total rows :  998
y_test_total rows :  998


In [124]:
CV_Accuracy=dict()#making the dict. 
for k in range(1,30):
    knn=KNeighborsClassifier(n_neighbors=k,algorithm='kd_tree')#calling the Knn classifiers. 
    knn.fit(x_train,y_train)#fiting with Dtrain data 
    pred=knn.predict(x_cv)#predicting the label values with CV data
    acc=accuracy_score(y_cv,pred)*float(100) #finding the accurarcy.
    CV_Accuracy[k]=acc#storing the key valye pairs of K and accuracy. 
    
#getting the optimal Value    
optimal_k=max(CV_Accuracy,key=CV_Accuracy.get)

#calling the Knn classifier with optimal value.
knn=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='kd_tree')
knn.fit(x_train,y_train)#fiting with Dtest data
pred=knn.predict(x_test)#predicting the label values with Dtest
acc=accuracy_score(y_test,pred)*float(100)#finding the accuracy. 
print("The accuracy of the model for Cross validation optimal-K {0} is {1} ".format(optimal_k,acc))

The accuracy of the model for Cross validation optimal-K 13 is 84.96993987975952 


In [132]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y_test,pred,pos_label=2)

auc=metrics.auc(fpr, tpr)

print(thresholds)

[2 1 0]


# [6] Conclusions

In [0]:
# Write few sentance about the results that you got and observation that you did from the analysis